In [1]:
import tensorflow as tf
import torch
from transformers import TFAutoModelForSequenceClassification, BertTokenizer
#import nltk
#from nltk.tokenize import word_tokenize
#nltk.download('punkt

import pandas as pd 
import keras
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

2024-07-11 18:39:40.727260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 18:39:40.757827: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 18:39:40.757860: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-11 18:39:40.776754: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 18:39:41.585344: W tensorflow/compiler/tf

In [2]:
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)

/home/rpierson/anaconda3/envs/secondenvi/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/rpierson/anaconda3/envs/secondenvi/lib/python3.11/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-07-11 18:39:46.905988: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10532 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2024-07-11 18:39:46.907093: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 

In [3]:
label_encoder = LabelEncoder()

In [4]:
df = pd.read_csv(f'/home/rpierson/Topic_Files/topic_0.csv')
df['Priority'] = label_encoder.fit_transform(df['Priority'])

for x in range(len(df)):
    if pd.isna(df.iloc[x]["Combined_Text"]):
        df.at[x, "Combined_Text"] = " "
        
count = 0
for x in range(len(df)):
    if pd.isna(df.iloc[x]["Combined_Text"]):
        count += count
        
count

0

In [5]:
df.tail()

,Combined_Text,Priority
1025,add new versions of equinox.jetty; orbit bundl...,2
1026,Help files doesnt open User Assistance Build I...,2
1027,compile error in pde api module from the sourc...,2
1028,compile error with org.eclipse.help.base due t...,2
1029,upgrade to Orbit stable build S20080427194908 ...,2


In [6]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [7]:
import re
def tokenize_function(example):
    return tokenizer(example["Combined_Text"], padding="max_length", truncation=True)
def remove_links(text):
    return re.sub(r'http\S+', '', text)

In [8]:
train_df['Combined_Text'] = train_df['Combined_Text'].apply(remove_links)
val_df['Combined_Text'] = val_df['Combined_Text'].apply(remove_links)
                                                        
                                                        

In [13]:

train_df['Combined_Text'] = tf.convert_to_tensor(train_df['Combined_Text'], dtype = tf.string)
train_df['Priority'] = tf.convert_to_tensor(train_df['Priority'], dtype = tf.int32)

val_df['Combined_Text'] = tf.convert_to_tensor(val_df['Combined_Text'], dtype = tf.string)
val_df['Priority'] = tf.convert_to_tensor(val_df['Priority'], dtype = tf.int32)

In [14]:
train_df

,Combined_Text,Priority
995,b'[website] Update links to IBM docs SWT ; Th...,2
507,b'Help Contents Frame Empty User Assistance Hi...,2
334,b'Spelling error in Ant Code Formatter prefere...,2
848,b'javadoc warning in build 200611140927 Compar...,2
294,b'Searching in help -> document not indexed m...,2
...,...,...
87,b'NL Fix for help_home.html User Assistance Th...,0
330,b'<input> tag Error. Ant ANT run..; ; <input> ...,2
466,b'[Help] Improve search by keyword User Assist...,3
121,b'[CVS Help] ref-46.htm in help docs is empty ...,1


In [15]:
val_df

,Combined_Text,Priority
31,b'Interfaces not exported User Assistance I tr...,2
109,b'Help uses xerces implementation classes inst...,2
136,b'duplicate options not handled the same as cm...,1
88,b'NL File errors Releng Please fix the followi...,0
918,b'Document the error handling in user and isv ...,2
...,...,...
482,b'eclipse build script is not a proper Bourne ...,2
545,b'extradir help topics not indexed when plugin...,2
110,b'Infocenter: servlet classes not found. User ...,1
514,b'Warnings in latest integration build User As...,2


In [10]:
#to tensorflow dataset


def df_to_dataset(df, label_column, shuffle=True, batch_size=32):
    df = df.copy()
    labels = df.pop(label_column)
    
    # Convert tokenized text to strings
    df['Combined_Text'] = df['Combined_Text'].apply(lambda x: ' '.join(x))
    
    # Convert each column to a tensor with appropriate dtype
    for col in df.columns:
        if col == 'Combined_Text':
            df[col] = df[col].apply(lambda x: tf.convert_to_tensor(str(x), dtype=tf.string))
        else:
            df[col] = df[col].apply(lambda x: tf.convert_to_tensor(x, dtype=tf.int32))
    
    # Create a dataset from the dictionary of tensors
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    
    return ds


In [11]:
train_dataset = df_to_dataset(train_df, 'Priority')
for batch in train_dataset.take(1):
    print(batch)
val_dataset = df_to_dataset(val_df, 'Priority', shuffle=False)

TypeError: sequence item 0: expected str instance, int found

In [ ]:
train_tokenized = train_df["Combined_Text"].apply(lambda x: tokenize_function({"Combined_Text": x}))
val_tokenized = val_df["Combined_Text"].apply(lambda x: tokenize_function({"Combined_Text": x}))

train_df["input_ids"] = train_tokenized.apply(lambda x: x["input_ids"])
train_df["attention_mask"] = train_tokenized.apply(lambda x: x["attention_mask"])
train_df["token_type_ids"] = train_tokenized.apply(lambda x: x["token_type_ids"])

val_df["input_ids"] = val_tokenized.apply(lambda x: x["input_ids"])
val_df["attention_mask"] = val_tokenized.apply(lambda x: x["attention_mask"])
val_df["token_type_ids"] = val_tokenized.apply(lambda x: x["token_type_ids"])

In [16]:
def one_hot_encode_label(features, label):
    num_classes = 5  # Change this to your actual number of classes
    one_hot_label = tf.one_hot(label, depth=num_classes)
    return features, one_hot_label

In [17]:
train_dataset = train_df.map(one_hot_encode_label)
val_dataset = val_df.map(one_hot_encode_label)

TypeError: one_hot_encode_label() missing 1 required positional argument: 'label'

In [ ]:
train_dataset = train_df.shuffle(len(train_df)).batch(16)
val_dataset = val_df.batch(16)

In [ ]:
optimizer = tf.keras.optimizers.Ftrl(learning_rate=0.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available. Number of GPUs:", torch.cuda.device_count())
    print("CUDA device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")
torch.cuda.set_device(torch.device("cuda:0"))

In [ ]:
if torch.cuda.is_available():
        print(f"LOG: Torch allocated Memory: \
                {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB", flush=True)
        print(f"LOG: Torch cached Memory: \
                {torch.cuda.memory_reserved() / 1024 ** 2:.2f} MB", flush=True)

if tf.config.experimental.list_physical_devices('GPU'):
    zero_info = tf.config.experimental.get_memory_info('GPU:0')
    print(f"LOG: TensorFlow, 0: Current memory usage: \
            {zero_info['current'] / 1024 ** 2:.2f} MB", flush=True)
    print(f"LOG: TensorFlow, 0: Peak memory usage: \
            {zero_info['peak'] / 1024 ** 2:.2f} MB", flush=True)

In [ ]:
# TensorFlow GPU configuration
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#if physical_devices:
#    try:
#        for device in physical_devices:
#            tf.config.experimental.set_memory_growth(device, True)
#        print(f"LOG: TensorFlow GPU devices: {physical_devices}.", flush=True)
#    except RuntimeError as exception:
#        print(f"LOG: TensorFlow GPU devices: {physical_devices}.", flush=True)
#        print(f"LOG: TensorFlow GPU configuration error: {exception}", flush=True)
#else:
#    print("ERROR: No TensorFlow GPU devices found.", flush=True)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
if torch.cuda.is_available():
        print(f"LOG: Torch allocated Memory: \
                {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB", flush=True)
        print(f"LOG: Torch cached Memory: \
                {torch.cuda.memory_reserved() / 1024 ** 2:.2f} MB", flush=True)

if tf.config.experimental.list_physical_devices('GPU'):
    zero_info = tf.config.experimental.get_memory_info('GPU:0')
    print(f"LOG: TensorFlow, 0: Current memory usage: \
            {zero_info['current'] / 1024 ** 2:.2f} MB", flush=True)
    print(f"LOG: TensorFlow, 0: Peak memory usage: \
            {zero_info['peak'] / 1024 ** 2:.2f} MB", flush=True)

from tensorflow.keras.optimizers import Adam
# model.compile(optimizer=Adam(3e-5))  # No loss argument!

model.compile(optimizer='Ftrl', loss='sparse_categorical_crossentropy', metrics = ['Accuracy', 'Precision', 'Recall'])


In [ ]:
#model.add(layers.Dense(5, activation='softmax'))

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_df['Priority']
)).shuffle(len(train_df)).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_df['Priority']
)).batch(16)

In [ ]:
model.fit(
    train_df,
    validation_data=val_df,
    epochs=15
)